### Test - Synthetic Diagnostic + Interferometry

</b> Author:   </b>  Louis Evans     </br>
</b> Reviewer: </b>  Stefano Merlini

### Import relevant files:

In [70]:
import sys
#add path
#sys.path.insert(0, '/Users/sm13118/Library/CloudStorage/OneDrive-ImperialCollegeLondon/dev/synthPy/')   # import path/to/synthpy
sys.path.insert(0, '../../solvers')
import numpy as np
import matplotlib.pyplot as plt
import vtk
from vtk.util import numpy_support as vtk_np
import matplotlib.pyplot as plt

#import solvers.full_solver as s
import full_solver as s
import importlib
importlib.reload(s)

<module 'full_solver' from '/home/administrator/Work/UROP_ICL_Internship/synthPy/examples/notebooks/../../solvers/full_solver.py'>

To use the ray tracer, first create a 'domain', load the domain with an electron distribution, calculate electron gradients, initialise a beam, and solve!
\
\
The output can either be plotted, or passed further through ray optics for synthetic diagnostics

In [71]:
# define some extent, the domain should be distributed as +extent to -extent, does not need to be cubic
extent_x = 5
extent_y = 5
extent_z = 10
n_cells  = 5

extent_x = extent_x / 1000
extent_y = extent_y / 1000
extent_z = extent_z / 1000

x = np.linspace(-extent_x, extent_x, n_cells)
y = np.linspace(-extent_y, extent_y, n_cells)
z = np.linspace(-extent_z, extent_z, n_cells)

probing_extent = extent_z
probing_direction = 'z'

domain = s.ScalarDomain(x = x, y = y, z = z, extent= probing_extent, probing_direction = probing_direction)     # create domain

[[[-0.005  -0.005  -0.005  -0.005  -0.005 ]
  [-0.005  -0.005  -0.005  -0.005  -0.005 ]
  [-0.005  -0.005  -0.005  -0.005  -0.005 ]
  [-0.005  -0.005  -0.005  -0.005  -0.005 ]
  [-0.005  -0.005  -0.005  -0.005  -0.005 ]]

 [[-0.0025 -0.0025 -0.0025 -0.0025 -0.0025]
  [-0.0025 -0.0025 -0.0025 -0.0025 -0.0025]
  [-0.0025 -0.0025 -0.0025 -0.0025 -0.0025]
  [-0.0025 -0.0025 -0.0025 -0.0025 -0.0025]
  [-0.0025 -0.0025 -0.0025 -0.0025 -0.0025]]

 [[ 0.      0.      0.      0.      0.    ]
  [ 0.      0.      0.      0.      0.    ]
  [ 0.      0.      0.      0.      0.    ]
  [ 0.      0.      0.      0.      0.    ]
  [ 0.      0.      0.      0.      0.    ]]

 [[ 0.0025  0.0025  0.0025  0.0025  0.0025]
  [ 0.0025  0.0025  0.0025  0.0025  0.0025]
  [ 0.0025  0.0025  0.0025  0.0025  0.0025]
  [ 0.0025  0.0025  0.0025  0.0025  0.0025]
  [ 0.0025  0.0025  0.0025  0.0025  0.0025]]

 [[ 0.005   0.005   0.005   0.005   0.005 ]
  [ 0.005   0.005   0.005   0.005   0.005 ]
  [ 0.005   0.005   0.00

Load some domain distribution

In [72]:
# load some external pvti
# import utils.handle_filetypes as load
# ne, dim, spacing = load.pvti_readin(filename)
# or load a test distribution

domain.test_exponential_cos()

Solve gradients, initialise beam, and solve

In [75]:
wl = 532e-9 #define laser wavelength

# initialise beam

#was 10000 by default, limited to 1000 as don't have access to HPC yet and am unsure of runtime, shall adjust later if able to realistically
Np = 10000    # number of photons
divergence = 5e-5   # realistic divergence value
beam_size = extent_x    # beam radius
ne_extent = probing_extent  # so the beam knows where to initialise initial positions
beam_type = 'circular'

initial_rays = s.init_beam(Np = Np, beam_size = beam_size, divergence = divergence, ne_extent = ne_extent, beam_type = beam_type, probing_direction = probing_direction)

# solve ray trace
domain.calc_dndr(wl)
print("Completed calc_dndr routine.")
final_rays = domain.solve(initial_rays)

Completed calc_dndr routine.
Starting ray trace.
Ray trace completed in:	 0.022309541702270508 s


Pass through ray optics: For Shadowgaphy, Schlieren, or Refractometry:

In [74]:
import rtm_solver as rtm

#in the diagnostic initialisation, details on the lens configurations, and detector dimensions can be specified
refractometer = rtm.RefractometerRays(final_rays)
refractometer.solve()
refractometer.histogram(bin_scale = 1, clear_mem = True)

sh=rtm.ShadowgraphyRays(final_rays)
sh.solve(displacement = 0)
sh.histogram(bin_scale = 1, clear_mem=True)

sh=rtm.SchlierenRays(final_rays)
sh.solve()
sh.histogram(bin_scale = 1, clear_mem=True)

#information accessed by .H(istogram) , e.g plt.imshow(refractometer.H)

ModuleNotFoundError: No module named 'sympy'

Synthetic Interferometry - This requires some extra steps in the ray tracing to include information on the phase and electric field values

In [ ]:
import solvers.full_solver as s

extent_x = 5
extent_y = 5
extent_z = 10

n_cells = 2

x = np.linspace(-extent_x, extent_x, n_cells)
y = np.linspace(-extent_y, extent_y, n_cells)
z = np.linspace(-extent_z, extent_z, n_cells)

probing_extent = extent_z
probing_direction = 'z'

domain = s.ScalarDomain(x = x, y = y, z = z, extent= probing_extent, probing_direction = probing_direction, phaseshift = True) #specify phaseshift TRUE

# load some external pvti


# import utils.handle_filetypes as load
# ne, dim, spacing = load.pvti_readin(filename)

# or load a test distribution

domain.test_exponential_cos()

wl = 532e-9 #define laser wavelength

# initialise beam

Np = 1e4 #number of photons
divergence = 5e-5 #realistic divergence value
beam_size = extent_x #beam radius
ne_extent = probing_extent #so the beam knows where to initialise initial positions
beam_type = 'circular'


initial_rays = s.init_beam(Np = Np, beam_size = beam_size, divergence = divergence, ne_extent = ne_extent, beam_type = beam_type, probing_direction = probing_direction)

# solve ray trace

domain.calc_dndr(wl)

final_rays, E = domain.solve(initial_rays, return_E = True)

n_fringes = 10
deg = 10 #adjust number of fringes and tilt in degrees from the vertical exis
final_E = s.interfere_ref_beam(final_rays, E, n_fringes, deg)

pattern=rtm.InterferometerRays(final_rays, E = final_E)
pattern.solve(wl = wl)
pattern.interferogram(bin_scale = 1, clear_mem=True) #use interferogram instead of histogram

#get interferogram histogram with sh.H


/Users/sm13118/Library/CloudStorage/OneDrive-ImperialCollegeLondon/dev/synthPy/solvers/full_solver.py:164: RuntimeWarning: overflow encountered in power
  self.ne = n_e0*10**(self.XX/s)*(1+np.cos(2*np.pi*self.YY/Ly))
/Users/sm13118/Library/Python/3.12/lib/python/site-packages/numpy/lib/function_base.py:1259: RuntimeWarning: invalid value encountered in subtract
  out[tuple(slice1)] = (f[tuple(slice2)] - f[tuple(slice3)]) / dx_0
/Users/sm13118/Library/Python/3.12/lib/python/site-packages/numpy/lib/function_base.py:1266: RuntimeWarning: invalid value encountered in subtract
  out[tuple(slice1)] = (f[tuple(slice2)] - f[tuple(slice3)]) / dx_n
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/scipy/interpolate/_rgi.py:504: RuntimeWarning: invalid value encountered in multiply
  term = np.asarray(self.values[edge_indices]) * weight[vslice]
/Users/sm13118/Library/CloudStorage/OneDrive-ImperialCollegeLondon/dev/synthPy/solvers/full_solver.py:265: RuntimeWarning: i

KeyboardInterrupt: 